# Synthetic population with progression fit and costs

This notebook simulates a synthetic patient population
using the fitted disease progression and cost model parameters
in order to investigate what-if scenarios.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from hem import simulation, costs, plotting
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pyprojroot import here

## Simulate and generate costs

In [ ]:
Q = np.genfromtxt(here() / "models" / "fitted_Q.csv", delimiter=",", skip_header=1)
Q

In [ ]:
target_means = pd.read_csv(here() / "models" / "expected_costs_by_state.csv",
                          index_col=0)
target_means.head()

In [ ]:
age_vec, costs_by_age, cost_names, costs_over_lifetime = costs.generate_patients_with_costs(Q, target_means)

# Delay state 3 entry

In [ ]:
# N.b. state 3 becomes state 2 with zero-indexing
state = 2
t_delay = 10
Q_delayed = simulation.get_intensity_matrix_for_delayed_state_entry(Q, state, t_delay)

In [ ]:
Q_delayed

In [ ]:
Q

In [ ]:
_, costs_by_age_delayed, _, costs_over_lifetime_delayed = costs.generate_patients_with_costs(Q_delayed, target_means)

## Plot results

In [ ]:
chart, plot_data = plotting.plot_simulated_cost_results(costs_by_age, cost_names)
chart

In [ ]:
chart, _ = plotting.plot_simulated_cost_results(costs_by_age_delayed, cost_names)
chart

## Total cost saving calculation

In [ ]:
mean_cost_saving_per_patient, var_cost_saving_per_patient = costs.calculate_mean_and_variance_of_cost_saving(costs_by_age, costs_by_age_delayed)

In [ ]:
mean_cost_saving_per_patient

In [ ]:
var_cost_saving_per_patient

In [ ]:
n_UK_HD_patients = 8_000
estimated_cost_saving_per_year_UK = mean_cost_saving_per_patient * n_UK_HD_patients

In [ ]:
estimated_cost_saving_per_year_UK

In [ ]:
variance_cost_saving_per_year_UK = var_cost_saving_per_patient * n_UK_HD_patients

In [ ]:
cost_saving_UK_95CI = [estimated_cost_saving_per_year_UK - 1.96 * np.sqrt(variance_cost_saving_per_year_UK),
                       estimated_cost_saving_per_year_UK + 1.96 * np.sqrt(variance_cost_saving_per_year_UK)]
cost_saving_UK_95CI